# __DOCKING MOLECULAR__
Autor: __Jesus Alvarado Huayhuaz__


Contacto: __inefable12@gmail.com__


-----------------------------------------------
Instalaremos Autodock $VINA$ y dockearemos en el target 

1. Ligando => formato pdbqt
2. Proteína => formato pdbqt
3. Generar el archivo de configuracion => texto

## __INSTALACION VINA__

In [ ]:
!wget https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz
!tar xzvf autodock_vina_1_1_2_linux_x86.tgz
%cd autodock_vina_1_1_2_linux_x86/bin

In [ ]:
!pwd
!ls

## __1. LIGANDOS__

Por lo general tenemos las moléculas en formato XYZ, 
smiles, sdf, etc. Entonces requerimos un convertidor
a PDBQT, podemos usar Open Babel

In [ ]:
!apt-get -qq install -y openbabel

In [ ]:
import os
file = open("molecula.smi", "w")
#file.write("O=C(N([C@@H](C(=O)NC1CCCCC1)c1cccnc1)c1ccc(C(C)(C)C)cc1)C1=NC=NC1" + os.linesep)
file.write("C1CCCC(O)C1(=O)N" + os.linesep)
file.write("C1CCCC(O)C1(C)N" + os.linesep)
file.write("C1CCCC(O)C1(CC)N" + os.linesep)
file.write("C1CCCC(O)C1(N)N")
file.close()

In [ ]:
!obabel -i smi molecula.smi -o sdf --gen3D > _.sdf
!obabel -i sdf _.sdf -o mol2 -O _.mol2 -h -m
!obabel *.mol2 -O opt.xyz --minimize --sd --steps 20000 --ff MMFF94 -p 7.4 --log -m
!obabel -i xyz *.xyz -o pdbqt -O lig_.pdbqt -m

In [ ]:
!rm *mol2 *smi *xyz *sdf

In [ ]:
# O puedes ingresar tu molécula ya en 3D y en cualquier formato 
from google.colab import files
files.upload() # open input type = file

In [ ]:
# Convirtiendo a PDBQT
!obabel -i pdb x77.pdb -o pdbqt -O lig_x77.pdbqt -m

## __PROTEINA__

In [ ]:
!wget http://www.rcsb.org/pdb/files/6W63.pdb.gz
!gzip -d 6W63.pdb.gz

Por practicidad insertar la proteína __ya "preparada" en Autodock Tools en formato PDBQT__

In [ ]:
#Instalación para observar moléculas en 3D 
!pip install py3Dmol

In [ ]:
# Visualizamos la proteína, sitio activo, aminoácidos de interés
import py3Dmol

def definegrid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({'center':{'x':bxi,'y':byi,'z':bzi},'dimensions': {'w':bxf,'h':byf,'d':bzf},'color':'blue','opacity': 0.6})

def viewprot(object,prot_PDBfile,resids):
  mol1 = open(prot_PDBfile, 'r').read()
  object.addModel(mol1,'pdb')
  object.setStyle({'cartoon': {'color':'spectrum'}})
  object.addStyle({'resi':resids},{'stick':{'colorscheme':'greenCarbon'}})

def viewprotgrid(prot_PDBfile,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10):
  mol_view = py3Dmol.view(1000,600)
  definegrid(mol_view,bxi,byi,bzi,bxf,byf,bzf)
  viewprot(mol_view,prot_PDBfile,resids)
  mol_view.setBackgroundColor('0xffffff')
  mol_view.zoomTo()
  mol_view.show() 

In [ ]:
from ipywidgets import interact,fixed,IntSlider
import ipywidgets
interact(viewprotgrid,
         prot_PDBfile = ['6W63.pdb'],
#Agrega los residuos que quieres visualizar
         resids = [(41,142,145)],
         bxi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         byi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bzi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bxf=ipywidgets.IntSlider(min=0,max=30, step=1),
         byf=ipywidgets.IntSlider(min=0,max=30, step=1),
         bzf=ipywidgets.IntSlider(min=0,max=30, step=1))

## __CONFIGURACION__

In [ ]:
# Para la proteína 6W63: lado 25, centro (-19.313,18.321,-27.425)
with open("configuracion","w") as f:
  f.write("#Archivo de configuracion \n")
  f.write("\n")
  f.write("#Opciones de entrada \n")
  f.write("receptor = 6W63_sin_ligando.pdbqt \n")
  #f.write("ligand = lig_opt_1.pdbqt \n")
  f.write("#flex = [residuos flexibles en el receptor en formato pdbqt] \n")
  f.write("#Configuracion del espacio de búsqueda \n")
#Tamaño de la caja 
  f.write("#Tamaño de la caja: bxf, byf, bzf) \n")
  f.write("size_x = 25 \n")
  f.write("size_y = 25 \n")
  f.write("size_z = 25 \n")
# Coordenadas del centro de la caja (ver)
  f.write("#Centro de la caja: bxi, byi, bzi \n")
  f.write("center_x = -19.313 \n")
  f.write("center_y = 18.321 \n")
  f.write("center_z = -27.425 \n")
  f.write("#Opciones de Salida \n")
  #f.write("out = conformaciones.pdbqt \n")
  f.write("#log = \n")
  f.write("\n")
  f.write("#Otras opciones \n")
  f.write("#cpu = 8 \n")
  f.write("exhaustiveness = 8 \n")
  f.write("num_modes = 1 \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")

In [ ]:
!grep -v '#' configuracion

## __EJECUTAMOS VINA__

In [ ]:
# Si queremos acoplar solo 1 molécula
!./vina --config configuracion 

Correr docking en __grupo__

In [ ]:
# Creamos un txt de los ligandos en formato pdbqt, teniendo cuidado con la proteina
!ls lig_* > inputs.txt
# Convertimos este txt en una lista
f = open("inputs.txt","r")
input_lig_pdbqt = f.read().splitlines()
print(input_lig_pdbqt)

In [ ]:
# Información de comandos de ejecución de VINA
#!./vina --help

In [ ]:
# Ejecutamos VINA
for i in input_lig_pdbqt:
  print(i)
  !./vina --ligand {i} --config configuracion --log {i}.log

In [ ]:
import glob
with open('resumen_VS.txt', 'w') as sm:
  for fn in sorted(glob.glob("./*.log")):
    with open(fn) as f:
      flag = False
      for line in f:
        if not flag and '----' in line:
          flag = True
        elif flag:
          flag = False
          cells = line.split()
          sm.write(fn + "    " + cells[1] + "\n")
          break

In [ ]:
!cat resumen_VS.txt